In [1]:
#!nvidia-smi
#!pip3 install -U tensorflow-gpu
!pip3 install transformers

In [40]:
import tensorflow as tf 
import pandas as pd 
import numpy as np 
import os 
import json 
from tqdm import tqdm
from transformers import BertTokenizer, TFBertForSequenceClassification,RobertaTokenizer, TFRobertaForSequenceClassification
from tensorflow.keras.layers import Softmax,Input
from tensorflow.keras.models import Model
from imblearn.over_sampling import RandomOverSampler
import tensorflow_datasets as tfds

from sklearn.utils import shuffle

In [52]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertForSequenceClassification: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['dropout_150', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [53]:
with open("nli_wordentail_bakeoff_data.json","r") as file:
    data=json.load(file)
    file.close()

In [54]:
print(data['train'][0:5])
print(len(data['dev']))

[[['abode', 'house'], 1], [['abortion', 'anaemia'], 0], [['abortion', 'aneurysm'], 0], [['abortion', 'blindness'], 0], [['abortion', 'deafness'], 0]]
2066


In [55]:
pairs=[x[0] for x in data['train']]
targets=[x[1] for x in data['train']]
sampler=RandomOverSampler(sampling_strategy="all",random_state=42)
pairs,targets=sampler.fit_resample(pairs,targets)


records=[[x[0],x[1]]for x in zip(pairs,targets)]
records=shuffle(records,random_state=0)
print(len(records))
data['train']=records
print(len(data['train']))

14000
14000


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [7]:
# ref=data['train'][0:5]
# data['train']=ref
# data['dev']=ref

In [56]:
def convert_example_to_feature(x):
    words=x.split("$")
    bert_input = tokenizer.encode_plus(
                        words[0],words[1],                      
                        add_special_tokens = True,
                        max_length = 30,
                        pad_to_max_length = True,
                        return_attention_mask = True,truncation=True)
    return bert_input

def map_example_to_dict(input_ids, attention_masks,token_type_ids,label):
    return {
      "input_ids": input_ids,
      "attention_mask": attention_masks,
      "token_type_ids":token_type_ids
    }, label

In [57]:
def encode_examples(ds, limit=-1):
  # prepare list, so that we can build up final TensorFlow dataset from slices.
  input_ids_list = []
  attention_mask_list = []
  token_type_ids_list=[]
  label_list = []
  if (limit > 0):
      ds = ds.take(limit)
    
  for review, label in tqdm(tfds.as_numpy(ds)):
    bert_input = convert_example_to_feature(review.decode())
    input_ids_list.append(bert_input['input_ids'])
    attention_mask_list.append(bert_input['attention_mask'])
    token_type_ids_list.append(bert_input['token_type_ids'])
    label_list.append([label])
  return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list,token_type_ids_list,label_list)).map(map_example_to_dict)

In [58]:
pairs=[x[0][0]+"$"+x[0][1] for x in data['train']]
targets=[x[1] for x in data['train']]
training_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            tf.cast(pairs, tf.string),
            tf.cast(targets, tf.int32)
        )
    )
)

pairs=[x[0][0]+"$"+x[0][1] for x in data['dev']]
targets=[x[1] for x in data['dev']]
testing_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            tf.cast(pairs, tf.string),
            tf.cast(targets, tf.int32)
        )
    )
)

In [59]:
train_ds = encode_examples(training_dataset).shuffle(10000).batch(128)
test_ds = encode_examples(testing_dataset).batch(128)

14000it [00:07, 1875.06it/s]
2066it [00:00, 2127.96it/s]


In [73]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Nadam(learning_rate=0.00001)
model.compile(loss=loss, optimizer=optimizer, metrics=[metric])
chkpnt=tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_accuracy:.4f}.h5',
                                              monitor='val_accuracy',
                                              verbose=1,
                                              mode='max',
                                              save_best_only=True)

In [74]:
model.config.hidden_dropout_prob=0.2
model.config.attention_probs_dropout_prob=0.2

In [75]:
model.fit(train_ds,epochs=20,validation_data=test_ds,callbacks=[chkpnt])

Epoch 1/20
110/110 [==============================] - ETA: 0s - loss: 0.0927 - accuracy: 0.9696
Epoch 00001: val_accuracy improved from -inf to 0.89400, saving model to model.01-0.8940.h5
110/110 [==============================] - 93s 846ms/step - loss: 0.0927 - accuracy: 0.9696 - val_loss: 0.3721 - val_accuracy: 0.8940
Epoch 2/20
110/110 [==============================] - ETA: 0s - loss: 0.0629 - accuracy: 0.9800
Epoch 00002: val_accuracy did not improve from 0.89400
110/110 [==============================] - 90s 816ms/step - loss: 0.0629 - accuracy: 0.9800 - val_loss: 0.4361 - val_accuracy: 0.8921
Epoch 3/20
110/110 [==============================] - ETA: 0s - loss: 0.0497 - accuracy: 0.9834
Epoch 00003: val_accuracy did not improve from 0.89400
110/110 [==============================] - 90s 817ms/step - loss: 0.0497 - accuracy: 0.9834 - val_loss: 0.4324 - val_accuracy: 0.8911
Epoch 4/20
110/110 [==============================] - ETA: 0s - loss: 0.0365 - accuracy: 0.9892
Epoch 00004:

In [79]:
model.load_weights('model.07-0.9042.h5')

In [80]:
preds=model.predict(test_ds)

In [81]:
original_values=[x[1] for x in data['dev']]

In [82]:
predicted_values=tf.math.argmax(preds[0],axis=1)
print(predicted_values.shape)

(2066,)


In [83]:
from sklearn.metrics import classification_report

print(classification_report(original_values,predicted_values))

              precision    recall  f1-score   support

           0       0.94      0.95      0.94      1732
           1       0.72      0.67      0.69       334

    accuracy                           0.90      2066
   macro avg       0.83      0.81      0.82      2066
weighted avg       0.90      0.90      0.90      2066

